In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.master("local[4]") \
.appName("Csv-Üzeri-SQL") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory","2g") \
.getOrCreate()

In [2]:
# We will take only data-time column


In [3]:
df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",";") \
.csv("C:\\Users\\alican\\Desktop\\BigData\\OnlineRetail.csv") \
.select("InvoiceDate").distinct()

In [4]:
df.show()

+----------------+
|     InvoiceDate|
+----------------+
| 3.12.2010 16:50|
| 7.12.2010 12:28|
| 8.12.2010 15:02|
|10.12.2010 09:53|
|12.12.2010 13:32|
|15.12.2010 13:21|
|16.12.2010 08:41|
|17.12.2010 09:52|
| 9.01.2011 11:43|
|11.01.2011 11:38|
|16.01.2011 15:50|
|25.01.2011 17:06|
|27.01.2011 12:10|
|28.01.2011 12:19|
|31.01.2011 12:16|
| 1.02.2011 10:04|
| 2.02.2011 11:16|
| 7.02.2011 15:01|
|11.02.2011 15:56|
|15.02.2011 13:52|
+----------------+
only showing top 20 rows



In [5]:
# Evet şimdi anlaşıldı. Format gün.ay.yıl saat:dakika yani dd.MM.yyyy HH:mm

# Datetime da ise varsayılan format yyyy-MM-dd HH:mm:ss



In [6]:
exits_format = 'dd.MM.yyyy HH:mm'

In [7]:
# SAMPLE DATE AND TIME OPERATIONS

In [15]:
from pyspark.sql import functions as F

df2 = df.withColumn("InvoiceDate", F.trim(F.col("InvoiceDate"))) \
.withColumn("normal_dates_struct", F.to_date(F.col("InvoiceDate"), exits_format)) \
.withColumn("standart_dates_struct", F.to_timestamp(F.col("InvoiceDate"), exits_format)) \


In [16]:
df2.show(10)

+----------------+-------------------+---------------------+
|     InvoiceDate|normal_dates_struct|standart_dates_struct|
+----------------+-------------------+---------------------+
| 3.12.2010 16:50|         2010-12-03|  2010-12-03 16:50:00|
| 7.12.2010 12:28|         2010-12-07|  2010-12-07 12:28:00|
| 8.12.2010 15:02|         2010-12-08|  2010-12-08 15:02:00|
|10.12.2010 09:53|         2010-12-10|  2010-12-10 09:53:00|
|12.12.2010 13:32|         2010-12-12|  2010-12-12 13:32:00|
|15.12.2010 13:21|         2010-12-15|  2010-12-15 13:21:00|
|16.12.2010 08:41|         2010-12-16|  2010-12-16 08:41:00|
|17.12.2010 09:52|         2010-12-17|  2010-12-17 09:52:00|
| 9.01.2011 11:43|         2011-01-09|  2011-01-09 11:43:00|
|11.01.2011 11:38|         2011-01-11|  2011-01-11 11:38:00|
+----------------+-------------------+---------------------+
only showing top 10 rows



In [17]:
df2.printSchema()

root
 |-- InvoiceDate: string (nullable = true)
 |-- normal_dates_struct: date (nullable = true)
 |-- standart_dates_struct: timestamp (nullable = true)



In [18]:
# CHANGING DATE FORMAT


In [21]:
format_tr = "dd/MM/yyyy HH:mm:ss"
format_eng = "MM-dd-yyyy HH:mm:ss"

df3 = df2 \
.withColumn("TIME_TR", F.date_format(F.col("standart_dates_struct"), format_tr)) \
.withColumn("TIME_ENG", F.date_format(F.col("standart_dates_struct"), format_eng)) \
.withColumn("unix_time", F.unix_timestamp(F.col("standart_dates_struct"))) \

df3.show(10)

+----------------+-------------------+---------------------+-------------------+-------------------+----------+
|     InvoiceDate|normal_dates_struct|standart_dates_struct|            TIME_TR|           TIME_ENG| unix_time|
+----------------+-------------------+---------------------+-------------------+-------------------+----------+
| 3.12.2010 16:50|         2010-12-03|  2010-12-03 16:50:00|03/12/2010 16:50:00|12-03-2010 16:50:00|1291387800|
| 7.12.2010 12:28|         2010-12-07|  2010-12-07 12:28:00|07/12/2010 12:28:00|12-07-2010 12:28:00|1291717680|
| 8.12.2010 15:02|         2010-12-08|  2010-12-08 15:02:00|08/12/2010 15:02:00|12-08-2010 15:02:00|1291813320|
|10.12.2010 09:53|         2010-12-10|  2010-12-10 09:53:00|10/12/2010 09:53:00|12-10-2010 09:53:00|1291967580|
|12.12.2010 13:32|         2010-12-12|  2010-12-12 13:32:00|12/12/2010 13:32:00|12-12-2010 13:32:00|1292153520|
|15.12.2010 13:21|         2010-12-15|  2010-12-15 13:21:00|15/12/2010 13:21:00|12-15-2010 13:21:00|1292

In [22]:
# ADDING DATE , DATE DIFFERENCE ,YEAR SEARCHING INTO TIMESTAMP


In [27]:
df4 = df2 \
.withColumn("one_year_added", F.date_add(F.col("standart_dates_struct"), 365)) \
.withColumn("year", F.year(F.col("standart_dates_struct"))) \
.withColumn("fark", F.datediff(F.col("one_year_added"), F.col("standart_dates_struct")))


In [28]:
df4.show(10)

+----------------+-------------------+---------------------+--------------+----+----+
|     InvoiceDate|normal_dates_struct|standart_dates_struct|one_year_added|year|fark|
+----------------+-------------------+---------------------+--------------+----+----+
| 3.12.2010 16:50|         2010-12-03|  2010-12-03 16:50:00|    2011-12-03|2010| 365|
| 7.12.2010 12:28|         2010-12-07|  2010-12-07 12:28:00|    2011-12-07|2010| 365|
| 8.12.2010 15:02|         2010-12-08|  2010-12-08 15:02:00|    2011-12-08|2010| 365|
|10.12.2010 09:53|         2010-12-10|  2010-12-10 09:53:00|    2011-12-10|2010| 365|
|12.12.2010 13:32|         2010-12-12|  2010-12-12 13:32:00|    2011-12-12|2010| 365|
|15.12.2010 13:21|         2010-12-15|  2010-12-15 13:21:00|    2011-12-15|2010| 365|
|16.12.2010 08:41|         2010-12-16|  2010-12-16 08:41:00|    2011-12-16|2010| 365|
|17.12.2010 09:52|         2010-12-17|  2010-12-17 09:52:00|    2011-12-17|2010| 365|
| 9.01.2011 11:43|         2011-01-09|  2011-01-09 11: